In [1]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd

from tqdm import tqdm
import requests
import io
import json

In [2]:
url = 'https://ckan.dataplatform.nl/api/3/action/package_show?id=9cc4de28-6d03-4b59-8c66-085b3e8b3956&include_tracking=true'
r = requests.get(url)
data = r.json()

csv_urls = [x['url'] for x in data['result']['resources']]
test_csv_urls = csv_urls[:3]


In [3]:
test_csv_urls

['https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/a12b08b4-d0bf-45f9-b17c-20448369754c/download/resource_2020_05_11_2020_05_18.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/df752c8d-a9a6-4f94-b1ee-2d24e623c343/download/resource_2020_05_04_2020_05_11.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/b9cf1bc9-d348-4663-9ab1-c4bab32e8bbb/download/resource_2020_04_27_2020_05_04.csv']

In [13]:
response = requests.get(test_csv_urls[2])
file_object = io.StringIO(response.content.decode('utf-8'))


In [14]:
df = pd.read_csv(file_object, usecols=['sensor', 'air_quality_observed_id', 'lon', 'lat',
     'recording_time', 'trip_sequence', 'humidity', 'pm2_5', 'pressure', 'temperature'])
df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")

In [15]:
df['pm2_5'][df['pm2_5'] < 1]

299       0
369       0
716       0
725       0
735       0
         ..
197539    0
197902    0
199190    0
199542    0
199670    0
Name: pm2_5, Length: 4961, dtype: int64

In [16]:
df['pm2_5'].mean()

8.569077845389227

In [23]:
df['pm2_5'].mean()

8.593882969414848

In [ ]:
# Download each .csv-file, pre-process, and save.
for csv in tqdm(test_csv_urls): 
    response = requests.get(csv)
    file_object = io.StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(file_object, usecols=['sensor', 'air_quality_observed_id', 'lon', 'lat',
     'recording_time', 'trip_sequence', 'humidity', 'pm2_5', 'pressure', 'temperature'])
    df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")


    # Remove all measurements with pm2.5 >150 ug/m3.
    df = df[~((df["pm2_5"]<0.5)|(df["pm2_5"]>150))]

    # Set all measurements with pm2.5 <1 ug/m3 to 1.
    df[df['pm2_5'] < 1] = 1